In [14]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from scipy import stats
import ipy_table as tbl
from itertools import islice

run_id = 181
qid = 1281

In [15]:
from openpyxl import Workbook, load_workbook

In [16]:
wb = load_workbook('../data/bibliometric nodes.xlsx')
ws = wb['useme']
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx, columns=cols)

In [18]:
df['id'] = df.index
df.head()

,label,timeset,weight<total_link_strength>,weight<norm._citations>,score<citations>,score<norm._citations>,score<pub._year>,weight<links>,weight<citations>,cluster,title,source,description,url,id
11,kamargianni (2013),None,162,1.3830,14,1.3830,2013,122,14,7,hybrid choice model to investigate effects of ...,transportation research record,"<table><tr><td>Authors:</td><td>kamargianni, m...",http://dx.doi.org/10.3141/2382-17,11
17,wang (2013),None,84,0.9878,10,0.9878,2013,61,10,3,policies and practices of low carbon city deve...,energy & environment,"<table><tr><td>Authors:</td><td>wang, c; lin, ...",None,17
23,shirmohammadi (2013),None,182,1.0866,11,1.0866,2013,118,11,1,analysis and design of tradable credit schemes...,transportation research record,"<table><tr><td>Authors:</td><td>shirmohammadi,...",http://dx.doi.org/10.3141/2333-04,23
28,jin (2013),None,168,0.4939,5,0.4939,2013,145,5,7,a recursive spatial equilibrium model for plan...,environment and planning b-planning & design,"<table><tr><td>Authors:</td><td>jin, y; echeni...",http://dx.doi.org/10.1068/b39134,28
43,schroeder (2013),None,80,0.8890,9,0.8890,2013,54,9,3,novel multisector networks and entrepreneurshi...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>schroeder, h; ...",http://dx.doi.org/10.1068/c3105ed,43


In [69]:
def get_topic(x):
    try:
        doi = x.replace('http://dx.doi.org/','')
        p = Doc.objects.get(
                wosarticle__di=doi,
                query=Query.objects.get(pk=qid),
                UT__icontains='WOS:'
            )
        topic = DocTopic.objects.filter(run_id=run_id,doc=p).order_by('-score').first().topic.title
    except:
        topic = ""
    return(topic)

In [70]:
df['main_topic'] = df.apply(lambda x: get_topic(x['url']),axis=1)

In [28]:
df.to_csv('../data/bibliometric_nodes.csv',index=False)

In [29]:
df

,label,timeset,weight<total_link_strength>,weight<norm._citations>,score<citations>,score<norm._citations>,score<pub._year>,weight<links>,weight<citations>,cluster,title,source,description,url,id,main_topic
11,kamargianni (2013),None,162,1.3830,14,1.3830,2013,122,14,7,hybrid choice model to investigate effects of ...,transportation research record,"<table><tr><td>Authors:</td><td>kamargianni, m...",http://dx.doi.org/10.3141/2382-17,11,"{travel, mode, choic}"
17,wang (2013),None,84,0.9878,10,0.9878,2013,61,10,3,policies and practices of low carbon city deve...,energy & environment,"<table><tr><td>Authors:</td><td>wang, c; lin, ...",None,17,
23,shirmohammadi (2013),None,182,1.0866,11,1.0866,2013,118,11,1,analysis and design of tradable credit schemes...,transportation research record,"<table><tr><td>Authors:</td><td>shirmohammadi,...",http://dx.doi.org/10.3141/2333-04,23,"{price, congest, road}"
28,jin (2013),None,168,0.4939,5,0.4939,2013,145,5,7,a recursive spatial equilibrium model for plan...,environment and planning b-planning & design,"<table><tr><td>Authors:</td><td>jin, y; echeni...",http://dx.doi.org/10.1068/b39134,28,"{model, network, optim}"
43,schroeder (2013),None,80,0.8890,9,0.8890,2013,54,9,3,novel multisector networks and entrepreneurshi...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>schroeder, h; ...",http://dx.doi.org/10.1068/c3105ed,43,
44,burch (2013),None,133,0.5927,6,0.5927,2013,81,6,3,novel multisector networks and entrepreneurshi...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>burch, s; schr...",http://dx.doi.org/10.1068/c1206,44,
45,aylett (2013),None,104,1.4817,15,1.4817,2013,72,15,3,networked urban climate governance: neighborho...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>aylett, a</td>...",http://dx.doi.org/10.1068/c11304,45,
46,killip (2013),None,59,1.0866,11,1.0866,2013,41,11,4,transition management using a market transform...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>killip, g</td>...",http://dx.doi.org/10.1068/c11336,46,
47,shey (2013),None,163,0.4939,5,0.4939,2013,86,5,3,building a municipal food policy regime in min...,environment and planning c-government and policy,"<table><tr><td>Authors:</td><td>shey, je; beli...",http://dx.doi.org/10.1068/c11235,47,
59,winters (2013),None,202,0.7903,8,0.7903,2013,107,8,5,mapping bikeability: a spatial tool to support...,environment and planning b-planning & design,"<table><tr><td>Authors:</td><td>winters, m; br...",http://dx.doi.org/10.1068/b38185,59,"{bicycl, cycl, cyclist}"


In [25]:
Doc.objects.filter(wosarticle__di='10.1016/j.enbuild.2012.10.004',scopus=False)

<QuerySet [<Doc: WOS:000314378500008>]>

In [52]:
cluster_topics=[]
nodois=0
dts = DocTopic.objects.filter(run_id=run_id)
for index, row in df.iterrows():
    if row['url'] is not None:
        doi = row['url'].replace('http://dx.doi.org/','')
        try:
            p = Doc.objects.get(
                wosarticle__di=doi,
                query=Query.objects.get(pk=qid),
                UT__icontains='WOS:'
            )
            tops = dts.filter(doc=p)
            for t in tops:
                cluster_topics.append({
                    "cluster": row['cluster'],
                    "topic": t.topic.title,
                    "score": t.score
                })
        except:
            nodois+=1

In [53]:
cts = pd.DataFrame.from_dict(list(cluster_topics))

cts.head()

,cluster,score,topic
0,7,0.005294,"{polici, instrument, effect}"
1,7,0.031409,"{bicycl, cycl, cyclist}"
2,7,0.005361,"{model, network, optim}"
3,7,0.054312,"{activ, school, health}"
4,7,0.011706,"{bus, servic, oper}"


In [62]:
ct_sum = cts.groupby(['cluster', 'topic']).agg({'score': 'sum'})
# Change: groupby state_office and divide by sum
ct_pcts = ct_sum.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
ct_pcts

In [66]:
g = ct_pcts['score'].groupby(level=0, group_keys=False)

res = g.apply(lambda x: x.order(ascending=False).head(5))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: FutureWarning: order is deprecated, use sort_values(...)
  app.launch_new_instance()


In [68]:
res.to_csv('../data/cluster_topics.csv')